In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
from pathlib import Path

In [2]:
db_path = Path("test.db")

if db_path.exists():
    db_path.unlink()

In [3]:
engine = create_engine("sqlite:///test.db")

In [4]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE IF NOT EXISTS test (a INT, b TEXT)"))

In [5]:
with engine.connect() as conn:
    conn.execute(text("INSERT INTO test (a, b) VALUES (1, 'foo')"))
    conn.execute(text("INSERT INTO test (a, b) VALUES (2, 'bar')"))
    conn.execute(text("INSERT INTO test (a, b) VALUES (3, 'baz')"))
    conn.commit()

In [6]:
with engine.connect() as conn:
    df = pd.read_sql_query("SELECT * FROM test", conn)

df

,a,b
0,1,foo
1,2,bar
2,3,baz


In [ ]:
# add new column to dataframe

df["c"] = [4, 5, 6]
df["d"] = ["qux", "quux", "quuz"]

df

,a,b,c
0,1,foo,4
1,2,bar,5
2,3,baz,6


In [8]:
from sqlmigrately import migrate_table

In [9]:
migrate_table("test", df, engine, push_data=True)

2024-11-23 18:56:25.928 | INFO     | sqlmigrately.main:migrate_table:21 - Detected new columns: [{'name': 'c', 'type': dtype('int64')}]
2024-11-23 18:56:25.929 | INFO     | sqlmigrately.utils:alter_table:80 - Executing sql: `ALTER TABLE test ADD COLUMN c int64`
2024-11-23 18:56:25.943 | INFO     | sqlmigrately.main:migrate_table:31 - Appending data to test


In [10]:
new_table = pd.read_sql_query("SELECT * FROM test", engine)

new_table

,a,b,c
0,1,foo,NaN
1,2,bar,NaN
2,3,baz,NaN
3,1,foo,4.0
4,2,bar,5.0
5,3,baz,6.0
